# ജനറേറ്റീവ് നെറ്റ്വർക്കുകൾ

റികറന്റ് ന്യൂറൽ നെറ്റ്വർക്കുകൾ (RNNs) மற்றும் അവയുടെ ഗേറ്റഡ് സെൽ വകഭേദങ്ങൾ, ഉദാഹരണത്തിന് ലോങ് ഷോർട്ട് ടേം മെമ്മറി സെലുകൾ (LSTMs) மற்றும் ഗേറ്റഡ് റികറന്റ് യൂണിറ്റുകൾ (GRUs), ഭാഷാ മോഡലിംഗിന് ഒരു സംവിധാനം നൽകുന്നു, അതായത് അവ വാക്കുകളുടെ ക്രമീകരണം പഠിച്ച് ഒരു സീക്വൻസിലെ അടുത്ത വാക്ക് പ്രവചിക്കാൻ കഴിയും. ഇതുവഴി RNN-കൾ **ജനറേറ്റീവ് ടാസ്കുകൾ**ക്കായി ഉപയോഗിക്കാം, ഉദാഹരണത്തിന് സാധാരണ ടെക്സ്റ്റ് ജനറേഷൻ, മെഷീൻ ട്രാൻസ്ലേഷൻ,甚至 ഇമേജ് ക്യാപ്ഷനിംഗ്.

മുൻ യൂണിറ്റിൽ ചർച്ച ചെയ്ത RNN ആർക്കിടെക്ചറിൽ, ഓരോ RNN യൂണിറ്റും അടുത്ത ഹിഡൻ സ്റ്റേറ്റ് ഔട്ട്പുട്ടായി ഉൽപ്പാദിപ്പിച്ചിരുന്നു. എന്നാൽ, ഓരോ റികറന്റ് യൂണിറ്റിനും മറ്റൊരു ഔട്ട്പുട്ട് കൂടി ചേർക്കാം, ഇത് ഒരു **സീക്വൻസ്** (മൂല സീക്വൻസിന്റെ നീളത്തിന് തുല്യമായ) ഔട്ട്പുട്ട് നൽകാൻ സഹായിക്കും. കൂടാതെ, ഓരോ ഘട്ടത്തിലും ഇൻപുട്ട് സ്വീകരിക്കാത്ത RNN യൂണിറ്റുകൾ ഉപയോഗിച്ച്, ചില പ്രാരംഭ സ്റ്റേറ്റ് വെക്ടർ എടുത്ത്, തുടർന്ന് ഔട്ട്പുട്ടുകളുടെ ഒരു സീക്വൻസ് ഉൽപ്പാദിപ്പിക്കാം.

ഈ നോട്ട്‌ബുക്കിൽ, നാം ടെക്സ്റ്റ് ജനറേറ്റ് ചെയ്യാൻ സഹായിക്കുന്ന ലളിതമായ ജനറേറ്റീവ് മോഡലുകളെ കേന്ദ്രീകരിക്കും. ലളിതത്വത്തിനായി, നാം **ക്യാരക്ടർ-ലെവൽ നെറ്റ്വർക്ക്** നിർമ്മിക്കാം, ഇത് അക്ഷരമുതൽ അക്ഷരം ടെക്സ്റ്റ് ജനറേറ്റ് ചെയ്യും. പരിശീലനത്തിനിടെ, നാം ഒരു ടെക്സ്റ്റ് കോർപ്പസ് എടുത്ത്, അത് അക്ഷര സീക്വൻസുകളായി വിഭജിക്കണം.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## അക്ഷര വാക്ക് നിർമ്മാണം

അക്ഷരനിര അടിസ്ഥാനത്തിലുള്ള ജനറേറ്റീവ് നെറ്റ്‌വർക്ക് നിർമ്മിക്കാൻ, വാക്കുകൾക്ക് പകരം ടെക്സ്റ്റ് വ്യക്തിഗത അക്ഷരങ്ങളായി വിഭജിക്കണം. ഇത് സാധ്യമാക്കാൻ വ്യത്യസ്തമായ ഒരു ടോക്കനൈസർ നിർവചിക്കാം:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


നമ്മുടെ ഡാറ്റാസെറ്റിൽ നിന്നുള്ള ടെക്സ്റ്റ് എങ്ങനെ എൻകോഡ് ചെയ്യാമെന്ന് ഒരു ഉദാഹരണം നോക്കാം:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## ജനറേറ്റീവ് RNN പരിശീലനം

RNN ഉപയോഗിച്ച് ടെക്സ്റ്റ് ജനറേറ്റ് ചെയ്യാൻ നാം സ്വീകരിക്കുന്ന മാർഗം ഇപ്രകാരമാണ്. ഓരോ ഘട്ടത്തിലും, നീളം `nchars` ഉള്ള ഒരു അക്ഷരക്രമം എടുത്ത്, ഓരോ ഇൻപുട്ട് അക്ഷരത്തിനും അടുത്ത ഔട്ട്പുട്ട് അക്ഷരം ജനറേറ്റ് ചെയ്യാൻ നെറ്റ്‌വർക്ക് ആവശ്യപ്പെടും:

!['HELLO' എന്ന വാക്ക് RNN ഉപയോഗിച്ച് ജനറേറ്റ് ചെയ്യുന്നതിന്റെ ഉദാഹരണം കാണിക്കുന്ന ചിത്രം.](../../../../../translated_images/ml/rnn-generate.56c54afb52f9781d.webp)

യഥാർത്ഥ സാഹചര്യത്തെ ആശ്രയിച്ച്, *end-of-sequence* `<eos>` പോലുള്ള ചില പ്രത്യേക അക്ഷരങ്ങൾ ഉൾപ്പെടുത്തേണ്ടതുണ്ടാകാം. നമ്മുടെ കേസിൽ, നാം അനന്തമായ ടെക്സ്റ്റ് ജനറേഷനായി നെറ്റ്‌വർക്ക് പരിശീലിപ്പിക്കാനാണ് ഉദ്ദേശിക്കുന്നത്, അതിനാൽ ഓരോ സീക്വൻസിന്റെയും വലിപ്പം `nchars` ടോക്കണുകളായി നിശ്ചയിക്കും. അതിനാൽ, ഓരോ പരിശീലന ഉദാഹരണവും `nchars` ഇൻപുട്ടുകളും `nchars` ഔട്ട്പുട്ടുകളും (ഇൻപുട്ട് സീക്വൻസ് ഒരു സിംബോളിന് ഇടത്തേക്ക് ഷിഫ്റ്റ് ചെയ്തതും) ഉൾക്കൊള്ളും. മിനിബാച്ച് ഇത്തരത്തിലുള്ള നിരവധി സീക്വൻസുകൾ അടങ്ങിയിരിക്കും.

മിനിബാച്ചുകൾ ജനറേറ്റ് ചെയ്യാനുള്ള മാർഗം, നീളം `l` ഉള്ള ഓരോ ന്യൂസ് ടെക്സ്റ്റും എടുത്ത് അതിൽ നിന്നുള്ള എല്ലാ സാധ്യമായ ഇൻപുട്ട്-ഔട്ട്പുട്ട് കോമ്പിനേഷനുകളും (ആകെ `l-nchars` കോമ്പിനേഷനുകൾ ഉണ്ടാകും) ജനറേറ്റ് ചെയ്യുകയാണ്. അവ ഒരു മിനിബാച്ച് രൂപപ്പെടും, കൂടാതെ ഓരോ പരിശീലന ഘട്ടത്തിലും മിനിബാച്ചുകളുടെ വലിപ്പം വ്യത്യസ്തമായിരിക്കും.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

ഇപ്പോൾ നമുക്ക് ജനറേറ്റർ നെറ്റ്‌വർക്ക് നിർവചിക്കാം. ഇത് മുൻവശം ചർച്ച ചെയ്ത ഏതെങ്കിലും റികറന്റ് സെല്ലിൽ അടിസ്ഥാനമാക്കാവുന്നതാണ് (സിംപിൾ, LSTM അല്ലെങ്കിൽ GRU). നമ്മുടെ ഉദാഹരണത്തിൽ നാം LSTM ഉപയോഗിക്കും.

നെറ്റ്‌വർക്ക് അക്ഷരങ്ങളെ ഇൻപുട്ടായി സ്വീകരിക്കുന്നതിനാൽ, വാക്കുകളുടെ വലുപ്പം വളരെ ചെറുതാണ്, അതിനാൽ എമ്പെഡ്ഡിംഗ് ലെയർ ആവശ്യമില്ല, ഒന്ന്-ഹോട്ട് എൻകോഡഡ് ഇൻപുട്ട് നേരിട്ട് LSTM സെല്ലിലേക്ക് പോകാം. എന്നാൽ, അക്ഷരങ്ങളുടെ നമ്പറുകൾ ഇൻപുട്ടായി നൽകുന്നതിനാൽ, LSTM-ലേക്ക് നൽകുന്നതിന് മുമ്പ് അവ ഒന്ന്-ഹോട്ട് എൻകോഡ് ചെയ്യേണ്ടതുണ്ട്. ഇത് `forward` പാസ്സിൽ `one_hot` ഫംഗ്ഷൻ വിളിച്ച് ചെയ്യുന്നു. ഔട്ട്പുട്ട് എൻകോഡർ ഒരു ലീനിയർ ലെയർ ആയിരിക്കും, ഇത് ഹിഡൻ സ്റ്റേറ്റ് ഒന്ന്-ഹോട്ട് എൻകോഡഡ് ഔട്ട്പുട്ടായി മാറ്റും.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

പരിശീലനത്തിനിടെ, നാം സൃഷ്ടിച്ച ടെക്സ്റ്റ് സാമ്പിൾ ചെയ്യാൻ ആഗ്രഹിക്കുന്നു. അതിനായി, നാം `generate` എന്ന ഫംഗ്ഷൻ നിർവചിക്കും, ഇത് ആരംഭിക്കുന്ന സ്ട്രിംഗ് `start` മുതൽ ആരംഭിച്ച് `size` നീളമുള്ള ഔട്ട്പുട്ട് സ്ട്രിംഗ് ഉത്പാദിപ്പിക്കും.

ഇത് പ്രവർത്തിക്കുന്ന വിധം ഇപ്രകാരം ആണ്. ആദ്യം, നാം മുഴുവൻ സ്റ്റാർട്ട് സ്ട്രിംഗ് നെറ്റ്‌വർക്കിലൂടെ കടത്തും, പിന്നീട് ഔട്ട്പുട്ട് സ്റ്റേറ്റ് `s`യും അടുത്ത പ്രവചിച്ച അക്ഷരം `out`യും എടുക്കും. `out` ഒന്ന്-ഹോട്ട് എൻകോഡുചെയ്തതിനാൽ, നാം `argmax` ഉപയോഗിച്ച് വാക്ക്‌സഭയിലെ അക്ഷരത്തിന്റെ ഇൻഡക്സ് `nc` കണ്ടെത്തും, പിന്നീട് `itos` ഉപയോഗിച്ച് യഥാർത്ഥ അക്ഷരം കണ്ടെത്തി ഫലമായ അക്ഷരങ്ങളുടെ പട്ടികയായ `chars`-ലേക്ക് ചേർക്കും. ഒരു അക്ഷരം സൃഷ്ടിക്കുന്ന ഈ പ്രക്രിയ ആവശ്യമായ അക്ഷരങ്ങളുടെ എണ്ണം `size` തവണ ആവർത്തിക്കും.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

ഇപ്പോൾ പരിശീലനം തുടങ്ങാം! പരിശീലന ലൂപ്പ് മുമ്പത്തെ എല്ലാ ഉദാഹരണങ്ങളിലെയും പോലെ തന്നെയാണ്, പക്ഷേ കൃത്യതയുടെ പകരം ഓരോ 1000 എപ്പോക്കിലും സാമ്പിൾ ചെയ്ത ജനറേറ്റഡ് ടെക്സ്റ്റ് പ്രിന്റ് ചെയ്യും.

നഷ്ടം (loss) കണക്കാക്കുന്ന രീതിക്ക് പ്രത്യേക ശ്രദ്ധ നൽകേണ്ടതാണ്. ഔട്ട്‌പുട്ട് `out` ഒന്ന്-ഹോട്ട്-എൻകോഡഡ് ഫോർമാറ്റിലായിരിക്കണം, കൂടാതെ പ്രതീക്ഷിച്ച ടെക്സ്റ്റ് `text_out` അക്ഷരങ്ങളുടെ ഇൻഡക്സുകളുടെ ലിസ്റ്റായിരിക്കണം. ഭാഗ്യവശാൽ, `cross_entropy` ഫംഗ്ഷൻ ആദ്യ аргументായി അനനോർമലൈസ്ഡ് നെറ്റ്‌വർക്ക് ഔട്ട്‌പുട്ട്, രണ്ടാം аргументായി ക്ലാസ് നമ്പർ സ്വീകരിക്കുന്നു, ഇത് നമുക്ക് ആവശ്യമായതും തന്നെയാണ്. കൂടാതെ ഇത് മിനിബാച്ച് വലുപ്പം അനുസരിച്ച് സ്വയം ശരാശരി കണക്കാക്കും.

പരിശീലനം വളരെ ദൈർഘ്യമാകാതിരിക്കാൻ `samples_to_train` സാമ്പിളുകൾ കൊണ്ട് പരിധി നിശ്ചയിച്ചിരിക്കുന്നു. നിങ്ങൾക്ക് കൂടുതൽ പരീക്ഷണങ്ങൾ നടത്താനും, ചില എപ്പോക്കുകൾക്കായി (അപ്പോൾ ഈ കോഡിന്റെ ചുറ്റും മറ്റൊരു ലൂപ്പ് സൃഷ്ടിക്കേണ്ടതുണ്ടാകും) ദൈർഘ്യമേറിയ പരിശീലനം നടത്താനും ഞങ്ങൾ പ്രോത്സാഹിപ്പിക്കുന്നു.


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

ഈ ഉദാഹരണം ഇതിനകം തന്നെ നല്ലൊരു ടെക്സ്റ്റ് സൃഷ്ടിക്കുന്നു, പക്ഷേ ഇത് പലവിധത്തിൽ മെച്ചപ്പെടുത്താൻ കഴിയും:
* **മിനിബാച്ച് ജനറേഷൻ മെച്ചപ്പെടുത്തൽ**. പരിശീലനത്തിനായി ഡാറ്റ തയ്യാറാക്കിയത് ഓരോ സാമ്പിളിൽ നിന്നുമൊരു മിനിബാച്ച് സൃഷ്ടിക്കുന്ന രീതിയിലാണ്. ഇത് അനുയോജ്യമല്ല, കാരണം മിനിബാച്ചുകൾ എല്ലാം വ്യത്യസ്ത വലുപ്പത്തിലുള്ളവയാണ്, ചിലത് സൃഷ്ടിക്കാനാകാത്തതും ഉണ്ടാകാം, കാരണം ടെക്സ്റ്റ് `nchars`-നേക്കാൾ ചെറുതാണ്. കൂടാതെ, ചെറിയ മിനിബാച്ചുകൾ GPU-യെ മതിയായ തോതിൽ ലോഡ് ചെയ്യാറില്ല. എല്ലാ സാമ്പിളുകളിലെയും വലിയൊരു ടെക്സ്റ്റ് ചങ്ക് എടുത്ത്, എല്ലാ ഇൻപുട്ട്-ഔട്ട്പുട്ട് ജോഡികളും സൃഷ്ടിച്ച്, അവ ഷഫിൾ ചെയ്ത്, സമാന വലുപ്പത്തിലുള്ള മിനിബാച്ചുകൾ സൃഷ്ടിക്കുന്നത് ബുദ്ധിമുട്ടില്ല.
* **മൾട്ടിലെയർ LSTM**. 2 അല്ലെങ്കിൽ 3 ലെയറുകളുടെ LSTM സെല്ലുകൾ പരീക്ഷിക്കുന്നത് ഉചിതമാണ്. മുൻ യൂണിറ്റിൽ പറഞ്ഞതുപോലെ, ഓരോ LSTM ലെയറും ടെക്സ്റ്റിൽ നിന്നുള്ള പ്രത്യേക പാറ്റേണുകൾ എടുക്കുന്നു, കറക്റ്റർ-ലെവൽ ജനറേറ്ററിന്റെ കാര്യത്തിൽ താഴത്തെ LSTM ലെയർ സില്ലബിളുകൾ എടുക്കുന്നതിന് ഉത്തരവാദിയാകും, മുകളിലെ ലെയറുകൾ വാക്കുകളും വാക്കുകളുടെ സംയോജനങ്ങളും എടുക്കുന്നതിന്. ഇത് LSTM കൺസ്ട്രക്ടറിലേക്ക് ലെയറുകളുടെ എണ്ണം പാരാമീറ്ററായി നൽകുന്നതിലൂടെ എളുപ്പത്തിൽ നടപ്പിലാക്കാം.
* നിങ്ങൾക്ക് **GRU യൂണിറ്റുകൾ** ഉപയോഗിച്ച് പരീക്ഷണങ്ങൾ നടത്താനും, ഏത് മെച്ചമാണ് എന്ന് കാണാനും, കൂടാതെ **വിവിധ ഹിഡൻ ലെയർ വലുപ്പങ്ങൾ** പരീക്ഷിക്കാനും ആഗ്രഹിക്കാം. വളരെ വലിയ ഹിഡൻ ലെയർ ഓവർഫിറ്റിംഗിന് കാരണമാകാം (ഉദാ: നെറ്റ്‌വർക്ക് കൃത്യമായ ടെക്സ്റ്റ് പഠിക്കും), ചെറിയ വലുപ്പം നല്ല ഫലം നൽകാതിരിക്കാം.


## സോഫ്റ്റ് ടെക്സ്റ്റ് ജനറേഷൻ ಮತ್ತು ടെംപറേച്ചർ

മുൻപ് `generate` എന്ന നിർവചനത്തിൽ, നാം എപ്പോഴും ഏറ്റവും ഉയർന്ന സാധ്യതയുള്ള അക്ഷരത്തെ അടുത്ത അക്ഷരമായി എടുത്തിരുന്നു. ഇതിന്റെ ഫലമായി, ടെക്സ്റ്റ് പലപ്പോഴും ഒരേ അക്ഷരക്രമങ്ങൾ വീണ്ടും വീണ്ടും "സൈക്ല്" ചെയ്യുന്നതായി കാണപ്പെട്ടു, ഉദാഹരണത്തിന്:
```
today of the second the company and a second the company ...
```

എങ്കിലും, അടുത്ത അക്ഷരത്തിനുള്ള സാധ്യത വിതരണത്തെ നോക്കിയാൽ, ഏറ്റവും ഉയർന്ന ചില സാധ്യതകളുടെ വ്യത്യാസം വലിയതല്ലായിരിക്കാം, ഉദാഹരണത്തിന് ഒരു അക്ഷരത്തിന് 0.2 സാധ്യതയുണ്ടെങ്കിൽ, മറ്റൊന്ന് 0.19 ആയിരിക്കാം. ഉദാഹരണത്തിന് '*play*' എന്ന ക്രമത്തിൽ അടുത്ത അക്ഷരം സ്പേസ് ആകാമോ, അല്ലെങ്കിൽ **e** (പദമായ *player* ൽപോലെ) ആകാമോ എന്നത് സമാനമായ സാധ്യതയുള്ളതാണ്.

ഇത് നമ്മെ ഈ നിഗമനത്തിലേക്ക് നയിക്കുന്നു: ഉയർന്ന സാധ്യതയുള്ള അക്ഷരം തിരഞ്ഞെടുക്കുന്നത് എപ്പോഴും "ന്യായമായ" കാര്യമല്ല, കാരണം രണ്ടാമത്തെ ഉയർന്ന സാധ്യതയുള്ള അക്ഷരം തിരഞ്ഞെടുക്കുന്നതും അർത്ഥവത്തായ ടെക്സ്റ്റിലേക്ക് നയിക്കാം. നെറ്റ്‌വർക്ക് ഔട്ട്പുട്ട് നൽകുന്ന സാധ്യത വിതരണത്തിൽ നിന്നാണ് അക്ഷരങ്ങൾ **സാമ്പിൾ** ചെയ്യുന്നത് കൂടുതൽ ബുദ്ധിമുട്ടുള്ളതും ഉചിതവുമാണ്.

ഈ സാമ്പിളിംഗ് `multinomial` ഫംഗ്ഷൻ ഉപയോഗിച്ച് ചെയ്യാം, ഇത്所谓的 **മൾട്ടിനോമിയൽ വിതരണം** നടപ്പിലാക്കുന്നു. താഴെ ഈ **സോഫ്റ്റ്** ടെക്സ്റ്റ് ജനറേഷൻ നടപ്പിലാക്കുന്ന ഫംഗ്ഷൻ നിർവചിച്ചിരിക്കുന്നു:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

നാം **temperature** എന്ന ഒരു പുതിയ പാരാമീറ്റർ പരിചയപ്പെടുത്തി, ഇത് ഏറ്റവും ഉയർന്ന സാധ്യതയോട് എത്രമാത്രം കർശനമായി പാലിക്കണമെന്ന് സൂചിപ്പിക്കാൻ ഉപയോഗിക്കുന്നു. temperature 1.0 ആണെങ്കിൽ, നാം ശരാശരി മൾട്ടിനോമിയൽ സാമ്പ്ലിംഗ് നടത്തുന്നു, temperature അനന്തതയിലേക്ക് പോകുമ്പോൾ - എല്ലാ സാധ്യതകളും സമാനമാകുന്നു, പിന്നെ നാം യാദൃച്ഛികമായി അടുത്ത അക്ഷരം തിരഞ്ഞെടുക്കുന്നു. താഴെ കൊടുത്ത ഉദാഹരണത്തിൽ temperature വളരെ കൂടുമ്പോൾ വാചകം അർത്ഥരഹിതമാകുന്നത് കാണാം, കൂടാതെ അത് 0-ന് അടുത്ത് പോകുമ്പോൾ "cycled" കർശനമായി സൃഷ്ടിച്ച വാചകത്തെപ്പോലെ തോന്നുന്നു.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ അധികാരപരമായ ഉറവിടമായി കണക്കാക്കപ്പെടണം. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
